In [6]:
import numpy as np

In [43]:
table_str = r'''
            Base-64 \Plus CE \Plus FT \Plus BERT & \textbf{.421} & \underline{.622} & \textbf{.502} & \underline{.203} & \textbf{.301} & \textbf{.243} \\
            Base-1132 \Plus CE & .266 & \textbf{.627} & .374 & .123 & \underline{.29} & .173 \\
            Base-364 \Plus CE \Plus BERT & .385 & .532 & .447 & .205 & .283 & .237 \\
            Base-832 \Plus CE \Plus FT & .307 & .592 & .404 & .146 & .283 & .193 \\
'''

In [44]:
table_rows = []
string_rows = {}

for i, table_row in enumerate(table_str.split('\n')):
    if table_row == '':
        continue
    
    if table_row.count('&') < 5:
        string_rows[i] = table_row
        continue
        
    table_rows.append(table_row)


In [45]:
def str_is_float(val):
    try:
        return float(val)
    except ValueError:
        return None

def clean_value(val):
    val = val.strip()
    
    if '\\textbf' in val or '\\underline' or r'\\' in val:
        val = val.replace('\\textbf{', '').replace('\\underline{', '').replace('}', '').replace(r'\\', '')
        
       
    float_val = str_is_float(val)
    if float_val is not None:
        return float_val
    else:
        return val
        
table_values = np.array([[clean_value(val) for val in table_row.split('&')[1:]] for table_row in table_rows if table_row.count('&') > 0])
table_rows_headers = [table_row.split('&')[0].strip() for table_row in table_rows]

In [46]:
max_values = []
second_values = []

for i in range(len(table_values[0])):
    current_items = table_values[:, i]
    max_index = np.argmax(current_items)
    max_value = np.max(current_items)
    max_values.append(max_value)
    
    mask = np.ones(current_items.shape, bool)
    mask[max_index] = False
    second_values.append(np.max(current_items[mask]))

In [47]:
def add_string_rows(current_result):
    while True:
        current_entries = current_result.count('\n') + 1
        if current_entries in string_rows.keys():
            current_result += f'{string_rows[current_entries]}\n'
        else:
            break
            
    return current_result

In [48]:
result = ''
for (table_row_header, row_values) in zip(table_rows_headers, table_values):
    result = add_string_rows(result)
    result += f'{table_row_header} & '
    
    for i, row_value in enumerate(row_values):
        if row_value == max_values[i]:
            result += r'\textbf{' + str(row_value)[1:] + '} & '
        elif row_value == second_values[i]:
            result += r'\underline{' + str(row_value)[1:] + '} & '
        else:
            result += f'{str(row_value)[1:]} & '
            
    result = result[:-2]
    result += r'\\' + '\n'
    
result = add_string_rows(result)
print(result)

Base-64 \Plus CE \Plus FT \Plus BERT & \textbf{.421} & \underline{.622} & \textbf{.502} & \underline{.203} & \textbf{.301} & \textbf{.243} \\
Base-1132 \Plus CE & .266 & \textbf{.627} & .374 & .123 & \underline{.29} & .173 \\
Base-364 \Plus CE \Plus BERT & \underline{.385} & .532 & \underline{.447} & \textbf{.205} & .283 & \underline{.237} \\
Base-832 \Plus CE \Plus FT & .307 & .592 & .404 & .146 & .283 & .193 \\

